In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import sys
# sys.path.append("../upliftnet")

In [ ]:
import logging
logging.getLogger('tensorflow').disabled = True

In [ ]:
import numpy as np
import keras
from keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt

import upliftnet.data
import upliftnet.net
import upliftnet.plots
import upliftnet.metrics

In [ ]:
import upliftnet.metrics

### Config

In [ ]:
# sample size
N_TREATMENT = 80_000
N_CONTROL = 20_000

# param dist
MU = [0, 0, 0]
COV = None

# ground truth
TREATMENT_TRUTH = [-4.5, -1.5, -0.8, 1.2]
CONTROL_TRUTH = [-5, -1.8, -0.8, -1.2]

# test set ration
TEST_RATIO = 0.8

### Synthetic dataset

In [ ]:
X, y, treatment = upliftnet.data.generate_logistic_data(
    treatment_coefs=TREATMENT_TRUTH,
    control_coefs=CONTROL_TRUTH,
    n_treatment=N_TREATMENT,
    n_control=N_CONTROL,
    mu=MU,
    cov=COV
)

In [ ]:
y.mean()

In [ ]:
indices = np.random.permutation(X.shape[0])
n_train = int(X.shape[0] * TEST_RATIO)
training_idx, test_idx = indices[:n_train], indices[n_train:]

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X[training_idx], y[training_idx], treatment[training_idx]))
    .shuffle(buffer_size=5000, reshuffle_each_iteration=True)
    .batch(1_000)
)

X_test, y_test, treatment_test = X[test_idx], y[test_idx], treatment[test_idx]

### Simple MLP model

In [ ]:
class UpliftMLP(upliftnet.net.UpliftNet):
    def __init__(self, hidden_sizes):
        super().__init__()
        self.dense = [
            layers.Dense(size, activation='relu')
            for size in hidden_sizes
        ]
        self.out = layers.Dense(1, activation='linear')
        
    def call(self, inputs):
        densei = self.dense[0](inputs)
        for dense in self.dense[1:]:
            densei = dense(densei)
        return self.out(densei)

uplift_model = UpliftMLP([32, 16])
uplift_model.compile(optimizer=keras.optimizers.Adam())

In [ ]:
res = uplift_model.fit(
    x=train_dataset,
    epochs=3
)

### Evaluate on test set

In [ ]:
preds_test = np.squeeze(uplift_model.predict(X[test_idx]), 1)

In [ ]:
upliftnet.plots.true_lift_plot(y_test, preds_test, treatment_test);

In [ ]:
upliftnet.plots.target_plot(y_test, preds_test, treatment_test);

In [ ]:
upliftnet.plots.cumulative_gain_plot(y_test, preds_test, treatment_test);